# Natural Language Processing 
### CS-UH 2216 - Spring 2019

## Sentiment Analysis of 100,000 IMDB movie reviews
---


### Downloading the Imdb movie review data set

In [6]:
import warnings
warnings.filterwarnings('ignore') 

import os

# The code below will check to see if the data directory exists; if not, it will download the data.
if os.path.exists("./aclImdb") == False:
    print("Downloading the Imdb movie review data set")
    !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar xf aclImdb_v1.tar.gz
#Shell command to show the files and directories we have under aclImdb
!ls aclImdb/*

aclImdb/README     aclImdb/imdb.vocab aclImdb/imdbEr.txt

aclImdb/test:
labeledBow.feat neg             pos             urls_neg.txt    urls_pos.txt

aclImdb/train:
labeledBow.feat pos             unsupBow.feat   urls_pos.txt
neg             unsup           urls_neg.txt    urls_unsup.txt


# 1. Analysis of data

In [50]:
#Run this cell first
print("number of positive training examples:")
pos_train = !ls aclImdb/train/pos/ | wc -l
pos_train = int(pos_train[0])
print(pos_train)
print("number of negative training examples:")
neg_train = !ls aclImdb/train/neg/ | wc -l
neg_train = int(neg_train[0])
print(neg_train)
print("number of unlabelled training examples:")
unl_train = !ls aclImdb/train/unsup/ | wc -l
unl_train = int(unl_train[0])
print(unl_train)
print("number of positive testing examples:")
pos_test = !ls aclImdb/test/pos/ | wc -l
pos_test = int(pos_test[0])
print(pos_test)
print("number of negative testing examples:")
neg_test = !ls aclImdb/test/pos/ | wc -l
neg_test = int(neg_test[0])
print(neg_test)

number of positive training examples:
12500
number of negative training examples:
12500
number of unlabelled training examples:
50000
number of positive testing examples:
12500
number of negative testing examples:
12500


#### 1.1 How many reviews are for training? 75000

In [44]:
print("total number of training reviews:" )
print(pos_train + neg_train + unl_train)

total number of training reviews:
75000


#### 1.2 How many reviews are for testing? 25000

In [45]:
print("total number of training reviews:" )
print(pos_test + neg_test)

total number of training reviews:
25000


#### 1.3 How many reviews are positive (in total in training and testing)? 25000

In [46]:
print("total positive instances in training and testing" )
print(pos_train + pos_test)

total positive instances in training and testing
25000


#### 1.4 How many reviews are negative (in total in training and testing)? 25000

In [47]:
print("total negative instances in training and testing" )
print(neg_train + neg_test)

total negative instances in training and testing
25000


#### 1.5 How many reviews are unlabelled (in total in training and testing)? 50000

In [49]:
print("total negative instances in training and testing" )
print(unl_train)

total negative instances in training and testing
50000


#### 1.6 What can we use unlabeled reviews for? 

We can use the unlabeled reviews for building unsupervised learning classification algorithms that can cluster and learn labels of reviews without being given the rating

#### 1.7 How was the positive/negative labeling done?

The positive and negative labels are classified based on the rating of the user reviews. A negative class is given to ratings <= 4 and a positive class is given to ratings >= 7 out of 10. Hence, reviews with more neural ratings are not included in the positive/negative sets

#### 1.8 Simply based on the labeling approach, do we expect some reviews to be harder than others for sentiment analysis?

Yes. This is because some reviews with the same label are closer to the extreme ends of the scale (1 ratings) while other reviews are closer to the neural end (4 rating). We would expect reviews with the same label but closer to the extreme end of the scale, to be easier to analyze.

#### 1.9 How many are the most negative review [1] (train and test)?

In [55]:
max_neg_test = !ls aclImdb/test/neg/ | grep "\w*1.txt" | wc -l
max_neg_test = int(max_neg_test[0])
max_neg_train = !ls aclImdb/train/neg/ | grep "\w*1.txt" | wc -l
max_neg_train = int(max_neg_train[0])
print("Total number of most negative reviews")
print(max_neg_test + max_neg_train)

Total number of most negative reviews
10122


#### 1.10 How many are the most positive reviews [10] (train and test)?

In [56]:
max_pos_test = !ls aclImdb/test/pos/ | grep "\w*10.txt" | wc -l
max_pos_test = int(max_pos_test[0])
max_pos_train = !ls aclImdb/train/pos/ | grep "\w*10.txt" | wc -l
max_pos_train = int(max_pos_train[0])
print("Total number of most positive reviews")
print(max_pos_test + max_pos_train)

Total number of most positive reviews
9731


### Load data to memory

In [58]:
import sklearn
from sklearn.datasets import load_files #load_files load text files with categories as subfolder names; 

# Directory of our data
traindir = r'./aclImdb/train'
testdir = r'./aclImdb/test'

# load pos/neg train and test data
train=load_files(traindir,categories=['pos','neg']) #load_files shuffles the text and categories by default.
test=load_files(testdir,categories=['pos','neg'])

# load an object with all the training data (positive, negative and unlabeled)
alltrain = load_files(traindir,categories=['pos','neg','unsup'])
#load_files return a dictionary-like object:
#1. 'data': the raw text data to learn
#2. 'target': the classification labels (integer index)
#3. 'target_names': the meaning of the labels
#4. 'filenames': the name of the file holding the data point

In [65]:
#Browse an example
#For example, if we want to see data point with index i 
i = 13374
print("Index  = %3d\n" % (i))
print("Text = %s\n" % (train.data[i]))
print("Label = %s\n" %(train.target_names[train.target[i]]))
print("Filename = %s\n" % (train.filenames[i]))

Index  = 13374

Text = b'Since this movie was based on a true story of a woman who had two children and was not very well-off, it was just scary as to how real it really was! The acting is what gave the movie that push to greatness.<br /><br />Diane Keaton portrayed the main character, Patsy McCartle who had two sons whom she adored. Her performance is what made the real life story come to life on a television screen. It was very hard to watch some of the scenes since they were so real as to what happens when one becomes addicted to drugs.<br /><br />Just watching this very loving mother go from sweet to not caring at all was hard, but so true. I have known people who have gone through withdrawl and it was very much like what happened in this movie, from what I remember.<br /><br />I also thought that it was very risky for the director to want to make a movie out of what happened to this woman. Yet it was done so well. I applaud the director for making this movie.<br /><br />I highly r

# 2. Investigating the quality of data